# Конфигурация для путей

In [1]:
import os

# === НАСТРАИВАЕМЫЕ ПУТИ ===
DATA_ROOT = "/kaggle/input/ai-chall-semi-final"  # корневая папка с данными

TRAIN_CSV_STAGE1 = os.path.join(DATA_ROOT, "train_stage_1.csv")
TRAIN_CSV_STAGE2 = os.path.join(DATA_ROOT, "train_stage_2.csv")
TRAIN_CSV_STAGE3 = os.path.join(DATA_ROOT, "train_stage_3.csv")

TEST_CSV_STAGE1 = os.path.join(DATA_ROOT, "test_stage_1.csv")
TEST_CSV_STAGE2 = os.path.join(DATA_ROOT, "test_stage_2.csv")
TEST_CSV_STAGE3 = os.path.join(DATA_ROOT, "test_stage_3.csv")
TEST_MAPPING_CSV = os.path.join(DATA_ROOT, "test_stage1_stage_2_mapping.csv")

# Пути к папкам с изображениями
TRAIN_STAGE1_IMG_DIR = os.path.join(DATA_ROOT, "train_stage_1", "train")
TRAIN_STAGE2_IMG_DIR = os.path.join(DATA_ROOT, "train_stage_2", "train_stage_2")
TRAIN_STAGE3_IMG_DIR = os.path.join(DATA_ROOT, "train_stage_3")

TEST_STAGE1_IMG_DIR = os.path.join(DATA_ROOT, "test_stage_1", "test")
TEST_STAGE2_IMG_DIR = os.path.join(DATA_ROOT, "test_stage_2", "test_stage_2")
TEST_STAGE3_IMG_DIR = os.path.join(DATA_ROOT, "test_stage_3")

# Путь для сохранения результатов
OUTPUT_DIR = "/kaggle/working"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Обучение

In [2]:
import cv2
import torch
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from torchmetrics.classification import BinaryAUROC, BinaryF1Score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
import os # Добавим импорт os для путей

warnings.filterwarnings("ignore")

# === ИЗВЛЕКАЕМ ID ВИДЕО ИЗ ПУТИ (оставим для совместимости, но используем для stage_3) ===
def extract_video_id(crop_path):
    # Для stage 3: FLUX.Context/e3999acd150296294c51c767ca4ce49e.png -> e3999acd150296294c51c767ca4ce49e
    if crop_path.split('/')[0] in ['FLUX.Context', 'REVE', 'MJ7']:
        parts = crop_path.split('/')
        if len(parts) >= 2:
             # Убираем расширение .png
            video_id = parts[-1].rsplit('.', 1)[0]
            return video_id
        else:
            # Если структура неожиданная, возвращаем сам путь как ID
            return crop_path
    else:
        # crop_path: DF40/train/real/09f6731cefbaa486f5f8a3597da2c7f4/053.png
        # video_id: 09f6731cefbaa486f5f8a3597da2c7f4
        parts = crop_path.split('/')
        return parts[-2]


# === ВСПОМОГАТЕЛЬНАЯ ФУНКЦИЯ: определяет путь к изображению ===
def get_image_path(crop_path):
    # crop_path: DF40/train/real/09f6731cefbaa486f5f8a3597da2c7f4/053.png
    if crop_path.startswith('DF40') or crop_path.startswith('VLDF'):
        # Это из train_stage_1
        return os.path.join(TRAIN_STAGE1_IMG_DIR, crop_path)
    elif crop_path.split('/')[0] in ['FLUX.Context', 'REVE', 'MJ7']:
        # Это из train_stage_3 (пути типа FLUX.Context/000001.png)
        return os.path.join(TRAIN_STAGE3_IMG_DIR, crop_path)
    else:
        # Это из train_stage_2
        return os.path.join(TRAIN_STAGE2_IMG_DIR, crop_path)


# === НОВЫЙ ДАТАСЕТ: возвращает ОДНО изображение за раз ===
class ImageDataset(Dataset): # Переименуем для ясности
    def __init__(self, df, transforms=None):
        self.transforms = transforms
        # Сбрасываем индекс, чтобы использовать его как уникальный ID для __getitem__
        self.df = df.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        crop_path = row['crop_path']
        label = row['label']

        img_path = get_image_path(crop_path)
        try:
            image = cv2.imread(img_path)
            if image is None:
                 print(f"Warning: Could not read image: {img_path}")
                 image = (np.random.rand(224, 224, 3) * 255).astype('uint8')
            else:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            print(f"Error loading: {img_path}, Error: {e}")
            image = (np.random.rand(224, 224, 3) * 255).astype('uint8')

        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        # Возвращаем одно изображение (C, H, W) и его метку
        # crop_path возвращаем для отладки/идентификации
        return image, torch.tensor(label, dtype=torch.float), crop_path


# === МОДЕЛЬ: ViT без агрегации ===
class ImageClassifier(pl.LightningModule):
    def __init__(self, model_name='vit_base_patch16_clip_224.openai'):
        super().__init__()
        self.backbone = timm.create_model(model_name=model_name, pretrained=True, num_classes=1,
                                         drop_rate=0.3, attn_drop_rate=0.1, drop_path_rate=0.1)
        self.loss_fn = torch.nn.BCEWithLogitsLoss()

        self.train_auroc = BinaryAUROC()
        self.val_auroc = BinaryAUROC()
        self.train_f1 = BinaryF1Score()
        self.val_f1 = BinaryF1Score()

    def forward(self, x):
        # x shape: (B, C, H, W) - батч изображений
        logits = self.backbone(x) # (B, 1)
        return logits.squeeze(-1) # (B,) - убираем последнюю размерность

    def training_step(self, batch, batch_idx):
        x, y, crop_paths = batch # batch[0]=x, batch[1]=y, batch[2]=crop_paths
        logits = self(x) # (B,)
        loss = self.loss_fn(logits, y) # (B,) vs (B,)

        probs = torch.sigmoid(logits)
        self.train_auroc.update(probs, y)
        self.train_f1.update(probs, y)

        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def on_train_epoch_end(self):
        self.log('train_auroc', self.train_auroc.compute(), prog_bar=True)
        self.log('train_f1', self.train_f1.compute(), prog_bar=True)
        self.train_auroc.reset()
        self.train_f1.reset()

    def validation_step(self, batch, batch_idx):
        x, y, crop_paths = batch
        logits = self(x) # (B,)
        loss = self.loss_fn(logits, y) # (B,) vs (B,)

        probs = torch.sigmoid(logits)
        self.val_auroc.update(probs, y)
        self.val_f1.update(probs, y)

        self.log('val_loss', loss, on_epoch=True, prog_bar=True)

    def on_validation_epoch_end(self):
        self.log('val_auroc', self.val_auroc.compute(), prog_bar=True)
        self.log('val_f1', self.val_f1.compute(), prog_bar=True)
        self.val_auroc.reset()
        self.val_f1.reset()

    def configure_optimizers(self):
        # backbone_params = [p for n, p in self.named_parameters() if 'backbone' in n and p.requires_grad]
        # other_params = [p for n, p in self.named_parameters() if 'backbone' not in n and p.requires_grad]
    
        # optimizer = torch.optim.AdamW([
        #     {'params': backbone_params, 'lr': 3e-6},
        #     {'params': other_params, 'lr': 3e-5}
        # ], weight_decay=1e-2)

        optimizer = torch.optim.AdamW(self.parameters(), lr=3e-6, weight_decay=1e-2)
    
        # Планировщик Cosine Annealing
        cosine_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, eta_min=1e-7, T_max=10-2) # T_max = epochs - 1 (warmup занимает 1 эпоху)
        
        # Планировщик Linear Warmup (на 1 эпоху)
        warmup_scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.1, end_factor=1.0, total_iters=2)
        
        # SequentialLR объединяет планировщики
        # milestones: список эпох, на которых переключаются планировщики.
        # milestones = [1] означает, что после 1 эпохи (индекс 0) переключится на второй планировщик.
        # Если max_epochs=10, то warmup будет на эпохе 0, а cosine начнётся с эпохи 1 и закончится на 9.
        combined_scheduler = torch.optim.lr_scheduler.SequentialLR(
            optimizer, 
            schedulers=[warmup_scheduler, cosine_scheduler], 
            milestones=[2] # Переключение после 2 эпохи (warmup)
        )
        
        return [optimizer], [{"scheduler": combined_scheduler, "interval": "epoch"}]

# === ЗАГРУЗКА И ОБЪЕДИНЕНИЕ ДАННЫХ ===
def load_and_combine_train_data():
    df1 = pd.read_csv(TRAIN_CSV_STAGE1)
    df2 = pd.read_csv(TRAIN_CSV_STAGE2)
    df3 = pd.read_csv(TRAIN_CSV_STAGE3) # Загружаем stage 3

    # Добавляем информацию о том, из какого этапа данные
    df1['stage'] = 'stage1'
    df2['stage'] = 'stage2'
    df3['stage'] = 'stage3'

    combined_df = pd.concat([
        df1[['crop_path', 'fake_type', 'label', 'stage']],
        df2[['crop_path', 'fake_type', 'label', 'stage']],
        df3[['crop_path', 'label', 'stage']] # Предполагаем, что в stage3 нет fake_type
    ], ignore_index=True)

    return combined_df

train_df = load_and_combine_train_data()

# --- ИСПОЛЬЗУЕМ ВАРИАНТ 2 (по video_id для stage_1/2) ---
stage12_df = train_df[train_df['stage'].isin(['stage1', 'stage2'])]
stage3_df = train_df[train_df['stage'] == 'stage3']

if not stage12_df.empty:
    stage12_df = stage12_df.copy() # Избегаем SettingWithCopyWarning
    stage12_df['video_id'] = stage12_df['crop_path'].apply(extract_video_id)
    # Уникальные video_id и их метки
    unique_videos_df = stage12_df.groupby('video_id')['label'].first().reset_index()
    train_video_ids, val_video_ids = train_test_split(
        unique_videos_df['video_id'],
        test_size=0.2,
        stratify=unique_videos_df['label'],
        random_state=42
    )
    # Фильтруем основной датафрейм
    train_df_stage12 = stage12_df[stage12_df['video_id'].isin(train_video_ids)]
    val_df_stage12 = stage12_df[stage12_df['video_id'].isin(val_video_ids)]
else:
    train_df_stage12 = pd.DataFrame(columns=stage3_df.columns)
    val_df_stage12 = pd.DataFrame(columns=stage3_df.columns)

# Stage 3 разбивается по crop_path, так как это отдельные изображения
train_df_stage3, val_df_stage3 = train_test_split(
    stage3_df,
    test_size=0.2,
    stratify=stage3_df['label'],
    random_state=42
)

# Объединяем
train_df_final = pd.concat([train_df_stage12, train_df_stage3], ignore_index=True)
val_df = pd.concat([val_df_stage12, val_df_stage3], ignore_index=True)

# Аугментации
CLIP_MEAN = [0.48145466, 0.4578275, 0.40821073]
CLIP_STD = [0.26862954, 0.26130258, 0.27577711]

train_transforms = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.OneOf([
        A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.4),
        A.CLAHE(clip_limit=2.0, p=0.2),
        A.RandomBrightnessContrast(p=0.4),
    ], p=0.4),
    A.Affine(rotate=(-15, 15), translate_percent=(0.05, 0.05), scale=(0.9, 1.1), shear=(-10, 10), p=0.3),
    A.OneOf([
        A.GaussNoise(std_range=(0.05, 0.15), p=0.3),
        A.MultiplicativeNoise(multiplier=(0.8, 1.2), p=0.2),
        A.ImageCompression(quality_range=(60, 95), p=0.5),
    ], p=0.3),
    # A.OneOf([
    #     A.MotionBlur(blur_limit=3, p=0.3),
    #     A.GaussianBlur(blur_limit=(3, 5), p=0.4),
    #     A.MedianBlur(blur_limit=5, p=0.3),
    # ], p=0.2),
    A.Normalize(mean=CLIP_MEAN, std=CLIP_STD),
    ToTensorV2()
])

val_transforms = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=CLIP_MEAN, std=CLIP_STD),
    ToTensorV2()
])

train_dataset = ImageDataset(train_df_final, transforms=train_transforms)
val_dataset = ImageDataset(val_df, transforms=val_transforms)

# batch_size > 1 теперь возможен
BATCH_SIZE = 128 # Установите желаемый размер батча
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# === Обучение ===
model = ImageClassifier(model_name='vit_base_patch16_clip_224.openai') # Убран параметр aggregation

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor='val_auroc',
    mode='max',
    save_top_k=1,
    filename='best-image-model-{epoch:02d}-{val_auroc:.4f}',
    verbose=True
)

early_stopping_callback = pl.callbacks.EarlyStopping(
    monitor='val_auroc',
    mode='max',
    patience=3,
    verbose=True
)

trainer = pl.Trainer(
    max_epochs=10,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1,
    precision="16-mixed",
    # accumulate_grad_batches=8, # accumulate_grad_batches больше не нужен, так как batch_size > 1
    gradient_clip_val=1.0,
    callbacks=[checkpoint_callback, early_stopping_callback]
)

trainer.fit(model, train_loader, val_loader)

best_model_path = checkpoint_callback.best_model_path
print(f"Лучшая модель сохранена: {best_model_path}")

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

pytorch_model.bin:   0%|          | 0.00/599M [00:00<?, ?B/s]

2025-11-08 14:49:19.747778: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762613359.899058      39 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762613359.944815      39 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Лучшая модель сохранена: /kaggle/working/lightning_logs/version_0/checkpoints/best-image-model-epoch=09-val_auroc=0.9960.ckpt


In [ ]:
!ls -la "/kaggle/working/lightning_logs/version_0/checkpoints/"

# Инференс

In [3]:
from tqdm.auto import tqdm # Используем tqdm.auto для лучшей совместимости
import os
import pandas as pd
import numpy as np
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# === ЗАГРУЗКА ЛУЧШЕЙ МОДЕЛИ ===
# Загрузка модели ImageClassifier
model = ImageClassifier.load_from_checkpoint(best_model_path) # Загружаем как ImageClassifier
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# === ЗАГРУЗКА ДАННЫХ ===
test_stage2_df = pd.read_csv(TEST_CSV_STAGE2)
mapping_df = pd.read_csv(TEST_MAPPING_CSV)

mapping_df['crop_path'] = mapping_df['crop_path'].str.lstrip('/')

print(f"Video IDs в test_stage_2.csv: {len(test_stage2_df)}")
print(f"Строк в mapping: {len(mapping_df)}")
print(f"Уникальных video_id в mapping: {mapping_df['video_id'].nunique()}")

video_to_paths = mapping_df.groupby('video_id')['crop_path'].apply(list).to_dict()

target_video_ids = test_stage2_df['video_id'].tolist()
print(f"Video IDs для инференса: {len(target_video_ids)}")

video_ids_in_mapping = [vid for vid in target_video_ids if vid in video_to_paths]
video_ids_not_in_mapping = [vid for vid in target_video_ids if vid not in video_to_paths]

print(f"Video_id из mapping: {len(video_ids_in_mapping)}")
print(f"Video_id НЕ из mapping: {len(video_ids_not_in_mapping)}")

val_transforms = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=CLIP_MEAN, std=CLIP_STD),
    ToTensorV2()
])

# === ИНФЕРЕНС ДЛЯ ВИДЕО (Stage 2, с использованием mapping для Stage 1)===
# Для каждой video_id загружаем все кадры, предсказываем вероятность для каждого,
# затем усредняем вероятности.
video_results = []

for video_id in tqdm(target_video_ids, desc="Processing videos"):
    frame_paths = []

    if video_id in video_to_paths:
        # Кадры из test_stage_1 (через mapping)
        frame_paths = [os.path.join(TEST_STAGE1_IMG_DIR, path) for path in video_to_paths[video_id]]
    else:
        # Кадры напрямую из test_stage_2
        video_dir = os.path.join(TEST_STAGE2_IMG_DIR, video_id)
        if os.path.exists(video_dir):
            frame_paths = [
                os.path.join(video_dir, f)
                for f in os.listdir(video_dir)
                if f.lower().endswith(('.png', '.jpg', '.jpeg'))
            ]
        else:
            print(f"Папка не найдена: {video_dir}")

    if not frame_paths:
        print(f"Нет кадров для video_id {video_id} → используем prob=0.5")
        prob = 0.5
        video_results.append({"video_id": video_id, "crop_path": None, "prob": prob})
        continue

    frame_probs = []
    for path in frame_paths:
        try:
            image = cv2.imread(path)
            if image is None:
                raise ValueError(f"Изображение не загружено: {path}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            print(f"Ошибка при загрузке {path}: {e}")
            image = (np.random.rand(224, 224, 3) * 255).astype('uint8')

        if val_transforms:
            transformed = val_transforms(image=image)
            image = transformed['image']

        # Подаем ОДНО изображение (C, H, W) -> (1, C, H, W)
        image_tensor = image.unsqueeze(0) # (1, C, H, W)
        image_tensor = image_tensor.to(device)

        with torch.no_grad():
            logits = model(image_tensor) # (1,) для ImageClassifier
            prob = torch.sigmoid(logits).cpu().item() # (1,) -> scalar
        frame_probs.append(prob)

    if frame_probs:
        avg_prob = sum(frame_probs) / len(frame_probs)
        prob = avg_prob
    else:
        prob = 0.5 # На всякий случай

    # Для видео: video_id заполнен, crop_path = None
    video_results.append({"video_id": video_id, "crop_path": None, "prob": prob})

# === СОХРАНЕНИЕ РЕЗУЛЬТАТОВ ДЛЯ ВИДЕО (временно, для отладки) ===
video_results_df = pd.DataFrame(video_results)
video_output_path = os.path.join(OUTPUT_DIR, "video_predictions_from_image_model.csv")
video_results_df.to_csv(video_output_path, index=False)
print(f"\n Инференс видео завершён. Обработано видео: {len(video_results)}")
print(f"Результаты видео сохранены в: {video_output_path}")

# === ИНФЕРЕНС ДЛЯ ОТДЕЛЬНЫХ КАДРОВ (Stage 1 и Stage 3) ===
def process_single_frames_csv(csv_path, img_dir):
    """
    Обрабатывает CSV как набор отдельных кадров.
    Возвращает список словарей для DataFrame.
    """
    df = pd.read_csv(csv_path)
    results = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing single frames from {csv_path.split('/')[-1]}"):
        crop_path = row['crop_path']
        # Определяем путь к изображению
        if crop_path.startswith('/'):
            img_path = os.path.join(TEST_STAGE1_IMG_DIR, crop_path.lstrip('/'))
        elif crop_path.split('/')[0] in ['FLUX.Context', 'REVE', 'MJ7']:
            img_path = os.path.join(TEST_STAGE3_IMG_DIR, crop_path)
        else:
            # Если структура неожиданная, пробуем оба пути
            img_path = os.path.join(img_dir, crop_path)
            if not os.path.exists(img_path):
                 # Пробуем как stage 1
                 img_path = os.path.join(TEST_STAGE1_IMG_DIR, crop_path.lstrip('/'))
                 if not os.path.exists(img_path):
                     # Пробуем как stage 3
                     img_path = os.path.join(TEST_STAGE3_IMG_DIR, crop_path)
                     if not os.path.exists(img_path):
                          print(f"Предупреждение: Не найден файл {crop_path}")
                          continue # Пропускаем, если не найден нигде

        try:
            image = cv2.imread(img_path)
            if image is None:
                raise ValueError(f"Изображение не загружено: {img_path}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            print(f"Ошибка при загрузке {img_path}: {e}")
            image = (np.random.rand(224, 224, 3) * 255).astype('uint8')

        if val_transforms:
            transformed = val_transforms(image=image)
            image = transformed['image']

        # Подаем ОДНО изображение (C, H, W) -> (1, C, H, W)
        image_tensor = image.unsqueeze(0) # (1, C, H, W)
        image_tensor = image_tensor.to(device)

        with torch.no_grad():
            logits = model(image_tensor) # (1,) для ImageClassifier
            prob = torch.sigmoid(logits).cpu().item() # (1,) -> scalar

        # Для отдельного кадра: video_id = None, crop_path заполнен
        results.append({"video_id": None, "crop_path": crop_path, "prob": prob})
    return results

# Запускаем инференс для Stage 1 (все кадры)
print("\nЗапуск инференса для ВСЕХ кадров из test_stage_1 (image-level)...")
single_frame_results_stage1 = process_single_frames_csv(TEST_CSV_STAGE1, TEST_STAGE1_IMG_DIR)

# Запускаем инференс для Stage 3
print("Запуск инференса для кадров из test_stage_3 (image-level)...")
single_frame_results_stage3 = process_single_frames_csv(TEST_CSV_STAGE3, TEST_STAGE3_IMG_DIR)

# === ОБЪЕДИНЕНИЕ ВСЕХ РЕЗУЛЬТАТОВ В SUBMISSION.CSV ===
all_results_df = pd.concat([
    video_results_df[['video_id', 'crop_path', 'prob']], # Результаты видео
    pd.DataFrame(single_frame_results_stage3),  # Результаты отдельных кадров Stage 3
    pd.DataFrame(single_frame_results_stage1) # Результаты отдельных кадров Stage 1
], ignore_index=True)

# Перемешиваем (опционально, для проверки структуры)
# all_results_df = all_results_df.sample(frac=1).reset_index(drop=True)

# === СОХРАНЕНИЕ ФИНАЛЬНОГО SUBMISSION.CSV ===
output_path = os.path.join(OUTPUT_DIR, "submission.csv")
all_results_df.to_csv(output_path, index=False)

print(f"\nИнференс завершён.")
print(f"Всего предсказаний: {len(all_results_df)}")
print(f"Предсказаний для видео: {all_results_df['video_id'].notna().sum()}")
print(f"Предсказаний для отдельных кадров: {all_results_df['crop_path'].notna().sum()}")
print(f"Результаты сохранены в: {output_path}")

# Выведем первые строки submission для проверки
print("\nПервые 10 строк submission.csv:")
print(all_results_df.head(10))

Video IDs в test_stage_2.csv: 894
Строк в mapping: 50629
Уникальных video_id в mapping: 1438
Video IDs для инференса: 894
Video_id из mapping: 614
Video_id НЕ из mapping: 280


Processing videos:   0%|          | 0/894 [00:00<?, ?it/s]


 Инференс видео завершён. Обработано видео: 894
Результаты видео сохранены в: /kaggle/working/video_predictions_from_image_model.csv

Запуск инференса для ВСЕХ кадров из test_stage_1 (image-level)...


Processing single frames from test_stage_1.csv:   0%|          | 0/50629 [00:00<?, ?it/s]

Запуск инференса для кадров из test_stage_3 (image-level)...


Processing single frames from test_stage_3.csv:   0%|          | 0/2700 [00:00<?, ?it/s]


Инференс завершён.
Всего предсказаний: 54223
Предсказаний для видео: 894
Предсказаний для отдельных кадров: 53329
Результаты сохранены в: /kaggle/working/submission.csv

Первые 10 строк submission.csv:
                           video_id crop_path      prob
0  009861907a17bad061979e7b9765b392      None  0.074718
1  00bad3570d8d7242b61a8dfebefb9ab9      None  0.361200
2  034098790c77b8ab99c16e31dd56430c      None  0.383149
3  04432fe7d856f595dfa4995486c2b8e1      None  0.723540
4  04909d6b9b736e1aae0d8e169b94547a      None  0.924547
5  05b91cd11e5c71d339545455ee7664e2      None  0.991933
6  0756089df197d50b478d5c3d39d24dea      None  0.471661
7  07ebce903fce6d3806b608ac69cc71c3      None  0.924918
8  083715d4f38f42363b4f01b009cc960d      None  0.654348
9  0914182d99bb41a626771b6619077ce3      None  0.539348


In [4]:
torch.save(model.state_dict(), "best_model.pth")

In [5]:
!ls -la "/kaggle/working/"

total 338516
drwxr-xr-x 4 root root      4096 Nov  8 17:28 .
drwxr-xr-x 5 root root      4096 Nov  8 14:47 ..
-rw-r--r-- 1 root root 343264215 Nov  8 17:28 best_model.pth
drwxr-xr-x 3 root root      4096 Nov  8 14:49 lightning_logs
-rw-r--r-- 1 root root   3305538 Nov  8 17:22 submission.csv
-rw-r--r-- 1 root root     47834 Nov  8 17:03 video_predictions_from_image_model.csv
drwxr-xr-x 2 root root      4096 Nov  8 14:48 .virtual_documents


In [6]:
from IPython.display import FileLink
FileLink("best_model.pth")

/kaggle/working/best_model.pth

# Про подбор гиперпараметров и архитектур:

На этапе проектирования архитектуры было проведено предварительное сравнение нескольких подходов, включая классические CNN-модели (ResNet, EfficientNet, Xception). Все они показали сопоставимые результаты на валидации — AUROC в диапазоне 0.80–0.85 (на public тесте), что указывает на их ограниченную способность улавливать тонкие артефакты генеративных моделей без специализированной обработки видео. Учитывая объём кода (а также время его выполнения) и фокус на основном решении, полные реализации этих моделей (для сравнения и анализа) не включены в финальный код, однако их результаты были зафиксированы и использованы для обоснованного выбора архитектуры.

В качестве основного подхода была выбрана Vision Transformer с CLIP-инициализацией, поскольку предобученные на мультимодальных данных ViT-модели демонстрируют повышенную чувствительность к аномалиям в текстурах и цветовых несогласованностях — ключевым признакам deepfake. Для агрегации по кадрам протестированы стратегии mean, max и обучаемое временное внимание, причём последнее показало наилучшую стабильность и качество.

Все гиперпараметры (learning rate для backbone и головы, вес регуляризации, тип аугментаций, dropout-ставки и др.) подбирались итеративно на основе валидационной AUROC, с учётом вычислительных ограничений и стабильности обучения. Также был проведён базовый анализ данных: проверена сбалансированность меток, распределение длин видео, и т.д.